In [ ]:
## https://github.com/vizsim/mapillary_coverage/issues/1

In [4]:
import pandas as pd
import geopandas as gpd
# Read the Parquet files


gdf_mp_pano = pd.read_parquet("germany_osm-highways_25-04-05_mp_pano_coverage_23-01-01until25-04-01_ratio.parquet")
gdf_mp_all = pd.read_parquet("germany_osm-highways_25-04-05_mp_all_coverage_23-01-01until25-04-01_ratio.parquet")


    osm_id      highway  mp_coverage_ratio  length_m_before_clip  \
0     1978      primary           1.000000             66.017066   
1  1880866  residential           0.019283            498.687025   
2  1880870  residential           0.020499            435.853088   
3  1881164  residential           0.040049            222.833428   
4  2293021         path           1.000000            109.656800   

   length_m_after_clip                                           geometry  
0            66.017066  b'\x01\x02\x00\x00\x00\x03\x00\x00\x00\x8d\x9b...  
1             9.615933  b'\x01\x02\x00\x00\x00\x03\x00\x00\x00\x98\xcd...  
2             8.934343  b'\x01\x02\x00\x00\x00\x03\x00\x00\x00k\xe3\x9...  
3             8.924277  b'\x01\x02\x00\x00\x00\x03\x00\x00\x00\x1c\x87...  
4           109.656800  b'\x01\x02\x00\x00\x00\x05\x00\x00\x00PT\xd8\x...  


In [8]:
gdf_mp_pano=gdf_mp_pano[["osm_id","mp_coverage_ratio"]].copy()

In [12]:
gdf_mp_pano_above_06=gdf_mp_pano[gdf_mp_pano["mp_coverage_ratio"]>=0.6].copy()

In [13]:
gdf_mp_pano_above_06["mapillary_coverage"] = "pano"

In [14]:
gdf_mp_pano_above_06

,osm_id,mp_coverage_ratio,mapillary_coverage
0,1978,1.00000,pano
4,2293021,1.00000,pano
9,2483295,0.65936,pano
10,2483296,1.00000,pano
17,2597457,0.97025,pano
...,...,...,...
469976,1375078594,1.00000,pano
469977,1375078595,1.00000,pano
469978,1375078596,1.00000,pano
469979,1375078597,1.00000,pano


In [15]:
gdf_mp_all=gdf_mp_all[["osm_id","mp_coverage_ratio"]].copy()

In [16]:
gdf_mp_all_above_06=gdf_mp_all[gdf_mp_all["mp_coverage_ratio"]>=0.6].copy()

In [17]:
gdf_mp_all_above_06

,osm_id,mp_coverage_ratio
2,104,1.000000
6,111,1.000000
7,117,1.000000
8,122,1.000000
9,123,0.874366
...,...,...
2693327,1375081655,1.000000
2693332,1375084788,0.603866
2693336,1375091830,1.000000
2693337,1375091831,1.000000


In [18]:
gdf_mp_all_above_06["mapillary_coverage"] = "regular"

In [29]:
both_concat=pd.concat([gdf_mp_pano_above_06,gdf_mp_all_above_06],ignore_index=True)

In [30]:
both_concat["mapillary_coverage"].value_counts()

mapillary_coverage
regular    1476231
pano        265933
Name: count, dtype: int64

In [ ]:
## drop duplicates, keep pano
both_concat = both_concat.sort_values(by="mapillary_coverage", ascending=True).drop_duplicates(subset="osm_id", keep="first")

In [32]:
both_concat["mapillary_coverage"].value_counts()

mapillary_coverage
regular    1210298
pano        265933
Name: count, dtype: int64

In [34]:
both_concat["osm_id"].value_counts()

osm_id
1375094702    1
1978          1
1079606439    1
1079606440    1
1079606441    1
             ..
1079629090    1
1079632827    1
1079632828    1
1079632832    1
1079606438    1
Name: count, Length: 1476231, dtype: int64

In [37]:
both_concat=both_concat[["osm_id","mapillary_coverage"]].copy()

In [39]:
both_concat

,osm_id,mapillary_coverage
0,1978,pano
177280,1079606439,pano
177281,1079606440,pano
177282,1079606441,pano
177283,1079606442,pano
...,...,...
758007,181567337,regular
758006,181567336,regular
758005,181567335,regular
758004,181567334,regular


In [41]:
both_concat.to_csv("output/germany_osm-highways_25-04-05_mp_coverage_23-01-01until25-04-01_ratio_above_06.csv",index=False)